<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## Deep dive analysis into a market

In this notebook we will have a deep dive into the market dynamics of a specific market. We will explore topics such as:
* Changes in demand and supply
* Share of total inventory with price cuts
* Location of the inventory and prevalence of price cuts
* Supply of inventory coming from new construction

#### What will you create in this notebook?

##### Understand Changes in Supply and Demand YoY
<p align="center">
  <img src="../../../images/changes_supply_yoy_Denver, CO.png" alt="Alt text">
</p>

##### Determine Share of Inventory With Price Cuts
<p align="center">
  <img src="../../../images/pct_inventory_price_reductions_line_chart_Denver, CO.png" alt="Alt text">
</p>

##### Visualize Location of Available Inventory
<p align="center">
  <img src="../../../images/map_active_listings_Denver, CO.png" alt="Alt text">
</p>

##### Visualize Price Cutting Activity
<p align="center">
  <img src="../../../images/map_active_listings_Denver, CO_with_price_reductions.png" alt="Alt text">
</p>

##### Understand available inventory by Property Type
<p align="center">
  <img src="../../../images/share_intentory_property_type_Denver, CO.png" alt="Alt text">
</p>

##### Determine Price Cuts by Property Type
<p align="center">
  <img src="../../../images/price_cuts_biweekly_intentory_property_type_Denver, CO.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/experimental/supply_and_demand/markets_that_could_disrupt.ipynb)

### 1. Import required packages and setup the Parcl Labs API key

In [1]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.ts_stats import TimeSeriesAnalysis
from parcllabs.beta.charting.utils import create_labs_logo_dict
from parcllabs.beta.charting.utils import (
    create_labs_logo_dict,
    save_figure,
    )
from parcllabs.beta.charting.styling import default_style_config as style_config


# Create a ParclLabsClient instance
client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=1000, 
    turbo_mode=True # set turbo mode to True
)

In [3]:
# We define the market and Parcl id for the USA
market_for_analysis_id = 2899750
us_parcl_id = 5826765
# we also define where do we want to save the outputs
ROOT_DIR = "../../../outputs" # Replace with your own directory

### Search for Markets 

In this notebook, we perform a deep dive into market dynamics for the Denver metropolitan area (MSA) with a Parcl ID of `5826765`. Additionally, we use the USA as a comparison point in one of the charts, which has already been defined above. However, you can conduct your own analysis by replacing the `parcl_id` with the ID of a market you are interested in exploring. Below, we have code that searches for the 100 largest markets in the country, including the USA.


In [4]:
# Retrieve top 100 metro markets, sorted by total population in descending order
metros = client.search.markets.retrieve(
    sort_by='TOTAL_POPULATION',  # Sort by total population
    sort_order='DESC',           # In descending order
    location_type='CBSA',        # Location type set to Core Based Statistical Area (CBSA)
    limit=100                    # Limit results to top 100 metros
)

# Retrieve national data for the United States to use as a benchmark
us = client.search.markets.retrieve(
    query='United States',  # Query for the United States as a whole
    limit=1                 # Limit results to one (national-level data)
)

# Concatenate metro market data with national data for comparison
markets = pd.concat([metros, us])

In [5]:
# Convert the unique 'parcl_id' values to a list, enabling queries for multiple markets if needed.
market_parcl_ids = markets['parcl_id'].tolist()

# Check if the market ID designated for analysis is within the list of market IDs.
# Print 'yes' if it is found.
if market_for_analysis_id in market_parcl_ids:
    print('yes')

# Retrieve the name of the market corresponding to 'market_for_analysis_id'.
# The query filters the 'markets' DataFrame by 'parcl_id' and extracts the market name. We will use this name for labeling purposes.
market_for_analysis_name = markets.query('parcl_id == @market_for_analysis_id')['name'].values[0]


yes


### Retrieve Supply and Demand Data

In [6]:
# Define the start date for supply and demand data
start_date = '2022-09-01'


# Retrieve the supply (for-sale inventory) data for the market starting from the specified date
supply_df = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=[market_for_analysis_id,us_parcl_id],
    auto_paginate=True,
    start_date=start_date,
)

# Retrieve the demand data (housing event counts) for the market starting from the specified date
demand_df = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=[market_for_analysis_id,us_parcl_id],
    auto_paginate=True,
    start_date=start_date,
)

# Retrieve the price data (housing event prices) for the market starting from January 2019
prices_df = client.market_metrics.housing_event_prices.retrieve(
    parcl_ids=[market_for_analysis_id,us_parcl_id],
    auto_paginate=True,
    start_date='2019-01-01',  # Different start date to capture historical price trends
)

data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01'}, params: {}
data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01'}, params: {}
data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2019-01-01'}, params: {}


In [7]:
# check length of each dataframe
print(f'Length of supply data: {len(supply_df)}, prices data: {len(prices_df)}, and demand data: {len(demand_df)}')

Length of supply data: 218, prices data: 136, and demand data: 48


In [8]:
# Check the date range of the data 
print(prices_df['date'].max())
print(demand_df['date'].max())
print(supply_df['date'].max())

2024-08-01 00:00:00
2024-08-01 00:00:00
2024-09-30 00:00:00


The `supply_df` dataframe contains all the inventory available for sale across all our markets. The `prices_df` dataframe contains information about the median price for sales, listings, and the standard deviation of prices. The `demand_df` dataframe provides details about the number of events that occurred in the market, including new listings, sales, and units offered for rent. This information constitutes the first step in our analysis, helping us understand the dynamics of supply and demand alongside price trends.

We also need information on price cuts. For this, we will use the `SDF`, specifically the `for_sale_market_metrics.for_sale_inventory_price_changes` method of our client. This endpoint will retrieve price cuts across all types of properties.


In [9]:
# Retrieve price changes in inventory for the market starting from the specified date, we use the parcl_id of the market and the US as a benchmark
price_changes_df = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    parcl_ids=[market_for_analysis_id,us_parcl_id],        # Specify the market by its parcl_id
    auto_paginate=True,
    start_date=start_date     # Use the same start date defined earlier for consistency
)

data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01'}, params: {}


In [10]:
# Output the length of the price changes DataFrame to verify the amount of data retrieved
print(f'Length of price changes data: {len(price_changes_df)}')

# Output the number of unique 'parcl_id' values in the price changes DataFrame to ensure market coverage
print(f'There are {len(price_changes_df.parcl_id.unique())} unique parcl_ids in the price changes data')

Length of price changes data: 218
There are 2 unique parcl_ids in the price changes data


Now that we have our data we can start our analysis.

### Supply and Demand imbalances

In [11]:
# Calculate monthly supply and percentage of price drops
# Supply data is bi-weekly, and price changes are weekly, so we resample both to a monthly frequency
supply_monthly = (
    supply_df.copy(deep=True)  # Create a deep copy of the supply DataFrame to avoid modifying the original data
    
    # Merge with price_changes_df on 'parcl_id' and 'date' and subset the data
    .merge(price_changes_df[['parcl_id', 'date', 'count_price_drop','median_pct_price_change']], 
           on=['parcl_id', 'date'])
    # Add new columns for percentage of price drops and resample dates to monthly
    .assign(
        pct_price_drops=lambda df: df['count_price_drop'] / df['for_sale_inventory'],  # Calculate percentage of price drops out of total suply
        date=lambda df: df['date'].dt.to_period('M').dt.to_timestamp()  # Convert the 'date' to monthly frequency
    )
    
    # Group the data by 'parcl_id' and 'date' (now monthly) and calculate the median
    .groupby(['parcl_id', 'date'])
    .agg({
        'for_sale_inventory': 'median',     # Calculate the median inventory for each market and month
        'pct_price_drops': 'median',         # Calculate the median percentage of price drops
        'median_pct_price_change': 'median' # Calculate the median percentage of price changes
    })
      # Filter the data for the market of interest
    # Reset the index to return a flat DataFrame
    .reset_index()

   )

In [12]:
# merge with demand data
supply_demand_data = (
    demand_df[['date', 'parcl_id', 'sales']]  # Select relevant columns from the demand DataFrame (date, parcl_id, and sales)
    .merge(supply_monthly,                    # Merge with the supply_monthly DataFrame that includes supply and price drop data
           on=['date', 'parcl_id'])           # Join on 'date' and 'parcl_id' to align data across markets and time periods
)
supply_demand_data.query('parcl_id == @market_for_analysis_id')

,date,parcl_id,sales,for_sale_inventory,pct_price_drops,median_pct_price_change
24,2024-08-01,2899750,7724,13875.0,0.464389,-2.005
25,2024-07-01,2899750,8179,14076.0,0.452970,-2.000
26,2024-06-01,2899750,8104,13540.5,0.405591,-1.975
27,2024-05-01,2899750,8922,12434.0,0.357495,-1.785
28,2024-04-01,2899750,8554,10494.0,0.317801,-1.670
29,2024-03-01,2899750,7519,8876.5,0.286267,-1.650
30,2024-02-01,2899750,6633,7421.5,0.288040,-1.625
31,2024-01-01,2899750,6071,5950.0,0.365514,-1.670
32,2023-12-01,2899750,8012,7325.0,0.453569,-1.930
33,2023-11-01,2899750,6690,9309.0,0.473397,-2.145


In [13]:
# Sort the DataFrame by 'parcl_id' and 'date' to ensure chronological order for percentage change calculations
supply_demand_imbalances = (
    supply_demand_data.copy(deep=True)  # Create a deep copy of the supply_demand_data DataFrame to avoid modifying the original data
    .sort_values(['parcl_id', 'date'])  # Sort by 'parcl_id' and 'date'
    
    .assign(
        # Calculate percentage change in 'sales' over 12 periods (1 year) for each 'parcl_id'
        pct_change_demand=lambda df: df.groupby('parcl_id')['sales'].pct_change(periods=12),
       
        # Calculate percentage change in 'for_sale_inventory' over 12 periods for each 'parcl_id'
        pct_change_supply=lambda df: df.groupby('parcl_id')['for_sale_inventory'].pct_change(periods=12),
        
        # Calculate a 3-month moving average of percentage change in demand ('pct_change_demand')
        ma_pct_change_demand=lambda df: df.groupby('parcl_id')['pct_change_demand']
                                           .transform(lambda x: x.rolling(window=3).mean()),
        
        # Calculate a 3-month moving average of percentage change in supply ('pct_change_supply')
        ma_pct_change_supply=lambda df: df.groupby('parcl_id')['pct_change_supply']
                                           .transform(lambda x: x.rolling(window=3).mean())
                        
        # Drop rows with missing values in the calculated columns
        )
    .dropna(subset=['pct_change_demand', 'pct_change_supply', 'ma_pct_change_demand', 'ma_pct_change_supply'])
    .assign(
        gap_demand_supply=lambda df: df['ma_pct_change_supply'] - df['ma_pct_change_demand']   
        )
    .sort_values('date', ascending=False)
    .merge(markets[['parcl_id', 'name']], on='parcl_id')
    .assign(
        state = lambda df: df['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0]),
        clean_name =lambda df: df.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
        )
    .query('parcl_id == @market_for_analysis_id')
    )

In [14]:
# Create the bar chart
# get the max date in month year format 
chart_max_date = supply_demand_imbalances['date'].max().strftime('%B %Y')
market_name = supply_demand_imbalances['clean_name'].iloc[0]
# Merge the gap data with the supply and demand data to ensure consistent x-values
merged_data = supply_demand_imbalances[['clean_name', 'ma_pct_change_demand', 'ma_pct_change_supply', 'gap_demand_supply', 'date']]
merged_data = merged_data.sort_values('date', ascending=True)

# Melt the data for the bar chart
data_for_bar = pd.melt(merged_data, 
                       id_vars=['date'], 
                       value_vars=['ma_pct_change_demand', 'ma_pct_change_supply'], 
                       var_name='type', 
                       value_name='percent_change')

data_for_bar['type'] = data_for_bar['type'].map({'ma_pct_change_demand': 'Demand', 
                                                 'ma_pct_change_supply': 'Supply',
                                                 })

fig = px.bar(data_for_bar, 
             x='date', 
             y='percent_change', 
             color='type', 
             barmode='group', 
             title=f'YoY Change in Supply and Demand in {market_name} as of {chart_max_date}',
             labels={'percent_change': 'Percent Change', 'clean_name': 'Market'},
             color_discrete_map={'Demand': 'red', 'Supply': 'green'})

# Update the legend names
for trace in fig.data:
    if trace.name == 'Demand':
        trace.name = 'Demand (Sales)'
    elif trace.name == 'Supply':
        trace.name = 'Supply (Inventory)'

# Define dimensions
CHART_WIDTH = 1600
CHART_HEIGHT = 800

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='Percent Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())
save_figure(fig, save_path=f'{ROOT_DIR}/changes_supply_yoy_{market_name}.png', 
            width=CHART_WIDTH, height=CHART_HEIGHT)

fig.show()

### Inventory with price reductions

In [15]:
# now lets do price reduction 
# Calculate the 3-period rolling average of price drops, filter using query, and extract parcl_ids
imbalanced_with_price_changes_data_all = (
    supply_monthly.copy(deep=True)
    .sort_values(by=['parcl_id', 'date'], ascending=[True, True])
    # Calculate the 3-month rolling average of price changes for each parcl_id
    .assign(
        ma_price_changes=lambda df: df.groupby('parcl_id')['pct_price_drops'].transform(lambda x: x.rolling(window=3).mean())
    )
    
    # Sort by the rolling average of price changes in descending order
    .sort_values(['parcl_id','date',], ascending=[True,True])
    .merge(markets[['parcl_id', 'name']], on='parcl_id')
    .assign(
        state = lambda df: df['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0]),
        clean_name =lambda df: df.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
        )
    .assign(clean_name =lambda df: np.where(df['parcl_id'] == us_parcl_id, 'USA', df['clean_name']))
    
)

print(f'length of imbalanced_with_price_changes_data_all is {len(imbalanced_with_price_changes_data_all)}')


length of imbalanced_with_price_changes_data_all is 50


In [16]:
# Get max date for chart
max_date_for_chart = imbalanced_with_price_changes_data_all['date'].max().date().strftime('%B %d, %Y')


CHART_WIDTH = 1600
CHART_HEIGHT = 800
# Create the line chart using Plotly Express
fig = px.line(
    imbalanced_with_price_changes_data_all,
    x='date',
    y='pct_price_drops',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_price_drops': '% of Inventory with Price Cuts'},
    title=f'Percentage of Inventory with Price Reductions for {market_name} as of ({max_date_for_chart})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(imbalanced_with_price_changes_data_all['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = imbalanced_with_price_changes_data_all[
        (imbalanced_with_price_changes_data_all['clean_name'] == trace.name) &
        (imbalanced_with_price_changes_data_all['date'] == latest_date)
    ]['pct_price_drops'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Price Reductions',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)
save_figure(fig, save_path=f'{ROOT_DIR}/pct_inventory_price_reductions_line_chart_{market_name}.png', 
            width=CHART_WIDTH, height=CHART_HEIGHT)

fig.show()

### Where is the supply of inventory located

In [17]:
# need to query properties with listing event in the Denver metro
dataframes = []
for property_type in ['SINGLE_FAMILY', 'CONDO', 'TOWNHOUSE','OTHER']:
    search_params = {
        'parcl_ids': [market_for_analysis_id],  # Required
        'property_type': property_type,  # Required
        'event_history_listing_flag': True,
        }
    properties =  client.property.search.retrieve(**search_params)
    dataframes.append(properties)


Processing Parcl IDs |████████████████████████████████████████| 1/1 [100%] in 21.9s (0.05/s) 
Processing Parcl IDs |████████████████████████████████████████| 1/1 [100%] in 2.2s (0.44/s) 
Processing Parcl IDs |████████████████████████████████████████| 1/1 [100%] in 1.7s (0.58/s) 
Processing Parcl IDs |████████████████████████████████████████| 1/1 [100%] in 1.9s (0.52/s) 


In [18]:
# transform the list of lists into a single list
properties_market = pd.concat(dataframes)
# check lenght
print(f'Length of properties_market is {len(properties_market)}')

Length of properties_market is 132286


In [19]:
# create a list of all properties to retrieve the listing events
property_ids = properties_market['parcl_property_id'].tolist()

In [20]:
# now get listing events in the last three months
# Define the parameters we want to use in the search for property events.
property_events_parameters = {
    'parcl_property_ids': property_ids,
    'event_type': 'LISTING',
    #'entity_owner_name': None, # Specify one of the options or None
    'start_date': '2023-07-01',
    #'end_date': '2021-01-01',
}

# Call the client with the list of property ids and the event_type as 'SALE' to retrieve the sale events for the properties.
# we can pass the search_params dictionary to the retrieve method to get the search results using **property_events_parameters
list_events = client.property.events.retrieve(
    **property_events_parameters
    )

print(f"Found {len(list_events)} events matching the criteria.")

Processing Parcl Property IDs |████████████████████████████████████████| 132286/132286 [100%] in 3.8s (34510.22/s) 
Found 180165 events matching the criteria.


In [21]:
# merge with properties_market to get the property details 
list_events_with_data = (
    list_events
    .merge(properties_market[['parcl_property_id','longitude','latitude','zip_parcl_id','property_type']], on='parcl_property_id')
)

In [22]:
#
df_map_listings = list_events_with_data.query("event_name=='LISTED_SALE' & event_date>='2024-08-01'")
print(len(df_map_listings))
# now group by parcl_property_id and get the latest listing event
df_map_listings = df_map_listings.sort_values('event_date', ascending=False).groupby('parcl_property_id').first().reset_index()
print(len(df_map_listings))

10944
10621


In [23]:
mbox_token = 'pk.eyJ1IjoiZGF0YXdyZXN0bGVyLXBhcmNsbGFicyIsImEiOiJjbHZ2bTRidGUxdndtMndvNnI5eGY5dDVoIn0.wXcsWmRjcDAlutloLezm5Q'

# Define a color map for property types (keeping original colors)
property_type_colors = {
    'SINGLE_FAMILY': '#ffa64d',
    'CONDO': 'blue',
    'TOWNHOUSE': '#ff3300',
    'OTHER': '#006600'
}

# Map the property type to the color
df_map_listings['color'] = df_map_listings['property_type'].map(property_type_colors)

# Create a scatter mapbox plot for each property type
traces = []
for property_type, color in property_type_colors.items():
    df_filtered = df_map_listings[df_map_listings['property_type'] == property_type]
    trace = go.Scattermapbox(
        lat=df_filtered['latitude'],
        lon=df_filtered['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,  # Map marker size
            color=color,
            opacity=0.3  # Keeping original opacity for map markers
        ),
        text=df_filtered['property_type'],
        hoverinfo='text',
        name=property_type,  # This will create a legend entry for each property type
        legendgroup=property_type,  # Group by property type
        showlegend=False  # Hide the legend for now
    )
    traces.append(trace)

# Create figure
fig = go.Figure(data=traces)

# Create "dummy" traces for the legend with improved visibility
for property_type, color in property_type_colors.items():
    fig.add_trace(go.Scattermapbox(
        lat=[None],  # No actual points on the map
        lon=[None],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=12,  # Larger size for legend symbols
            color=color,
            opacity=1.0  # Fully opaque for legend visibility
        ),
        name=property_type,  # Same name to ensure it appears in the legend
        showlegend=True
    ))

# Update layout
fig.update_layout(
    mapbox_accesstoken=mbox_token,
    mapbox=dict(
        center=dict(lat=df_map_listings['latitude'].median(), lon=df_map_listings['longitude'].median()),
        style="dark",
        zoom=9
    ),
    height = CHART_HEIGHT,
    width=CHART_WIDTH,
    title=dict(
        text=f"Active Listings in {market_name}",
        font=dict(size=24, family="Arial", weight='bold', color='white'),
        y=0.98,
        x=0.5,
        xanchor='center'
    ),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    font=dict(family="Arial", size=14, color="#ffffff"),
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict(
        title=dict(
            text="Property Types",
            font=dict(color="white", size=16)
        ),
        font=dict(color="white", size=14),
        bgcolor="rgba(0, 0, 0, 0.9)",  # Less transparent for better contrast
        bordercolor="rgba(255, 255, 255, 0.5)",  # Subtle border for separation
        borderwidth=1,
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        orientation='v',
        traceorder='normal',
    )
)

fig.add_layout_image(
    create_labs_logo_dict(color='white', x=0.2)
)

# Show the figure
fig.show()

# Save the figure
save_figure(fig, save_path=f'{ROOT_DIR}/map_active_listings_{market_name}.png',
            width=CHART_WIDTH, height=CHART_HEIGHT)


### Zip code Analysis

In [24]:
# get unique listing zip codes
list_of_zips_ids = properties_market['zip_parcl_id'].unique().tolist()

In [25]:
# get the list of supply
changes_supply_yoy_zips = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=list_of_zips_ids,
    auto_paginate=True,
    start_date='2023-01-01',
    )

data: {'parcl_id': ['5351403', '5350894', '5351184', '5571505', '5571508', '5571001', '5571019', '5571514', '5571045', '5351454', '5570891', '5351447', '5596239', '5571020', '5570866', '5351416', '5571522', '5596236', '5350914', '5571041', '5351405', '5570678', '5351214', '5596209', '5350886', '5351211', '5350881', '5570868', '5350883', '5432206', '5351194', '5351176', '5571507', '5351459', '5570859', '5570861', '5571519', '5351453', '5570681', '5350880', '5571531', '5596229', '5571551', '5571524', '5350879', '5351395', '5351452', '5571532', '5351195', '5571025', '5351192', '5351461', '5350893', '5351193', '5351207', '5571506', '5571011', '5596225', '5350878', '5571010', '5351206', '5350921', '5432208', '5571533', '5351485', '5571050', '5351209', '5596238', '5570873', '5596240', '5351408', '5432225', '5596215', '5571521', '5350929', '5351488', '5596231', '5351213', '5570878', '5351460', '5432220', '5350882', '5350896', '5571024', '5350903', '5571034', '5351200', '5571530', '5571027', '

In [26]:
# Retrieve market details for each ZIP code in the provided list.
zip_details = []
for zip in list_of_zips_ids:
    # Use the client to fetch market details for each ZIP code (parcl_id) in the list.
    # The 'auto_paginate=True' parameter ensures all paginated results are retrieved automatically.
    zip_df = client.search.markets.retrieve(
        parcl_id=zip,
        auto_paginate=True
    )
    # Append the DataFrame of the ZIP code details to the 'zip_details' list.
    zip_details.append(zip_df)

# Concatenate all individual ZIP code DataFrames into a single DataFrame.
zip_details = pd.concat(zip_details)


In [27]:
changes_supply_yoy_zips_data = (
    changes_supply_yoy_zips.copy(deep=True)
    # Sort the DataFrame by 'parcl_id' and 'date' to prepare for time series operations.
    .sort_values(['parcl_id', 'date'])
    # Convert 'date' to the start of each month to normalize the date field to monthly frequency.
    .assign(
        date=lambda df: df['date'].dt.to_period('M').dt.to_timestamp()
    )
    # Group by 'parcl_id' and 'date' to aggregate data by market and month.
    .groupby(['parcl_id', 'date'])
    .agg({
        'for_sale_inventory': 'median',  # Calculate the median 'for_sale_inventory' for each group.
    })
    .reset_index()
    # Re-sort by 'parcl_id' and 'date' after aggregation.
    .sort_values(['parcl_id', 'date'])
    .assign(
        # Calculate the inventory level from 12 months ago for each 'parcl_id'.
        inventory_12_months_ago=lambda df: df.groupby('parcl_id')['for_sale_inventory'].shift(12),
        # Calculate the percentage change in 'for_sale_inventory' over 12 months.
        pct_change_supply=lambda df: df.groupby('parcl_id')['for_sale_inventory'].pct_change(periods=12),
        # Calculate the absolute difference in inventory from 12 months ago.
        difference=lambda df: df['for_sale_inventory'] - df['inventory_12_months_ago']
    )
    # Calculate a 3-month moving average of the yearly percentage change in supply.
    .assign(
        ma_pct_change_supply_yearly=lambda df: df.groupby('parcl_id')['pct_change_supply']
                                                .transform(lambda x: x.rolling(window=3).mean())
    )
    # Filter data to include only records from September 2024 onwards.
    .query('date >= "2024-09-01"')
    # Sort data by 'difference' and 'pct_change_supply' in descending order.
    .sort_values(['difference', 'pct_change_supply'], ascending=[False, False])
    # Merge with 'zip_details' to include zip code names for each 'parcl_id'.
    .merge(zip_details[['parcl_id', 'name']], on='parcl_id')
    # Rename the 'name' column to 'zip_code' for clarity.
    .rename(columns={'name': 'zip_code'})
)

# Display the first 10 rows of the final DataFrame.
changes_supply_yoy_zips_data.head(10)


,parcl_id,date,for_sale_inventory,inventory_12_months_ago,pct_change_supply,difference,ma_pct_change_supply_yearly,zip_code
0,5571508,2024-09-01,239.0,97.0,1.463918,142.0,1.516470,80019
1,5350914,2024-09-01,294.0,169.0,0.739645,125.0,1.235796,80013
2,5571019,2024-09-01,305.0,188.0,0.622340,117.0,0.808541,80601
3,5351195,2024-09-01,256.0,142.5,0.796491,113.5,1.326483,80014
4,5351214,2024-09-01,292.0,188.0,0.553191,104.0,1.077948,80015
5,5350921,2024-09-01,285.0,192.5,0.480519,92.5,0.621692,80108
6,5571011,2024-09-01,462.0,373.5,0.236948,88.5,0.475995,80134
7,5351405,2024-09-01,231.0,148.5,0.555556,82.5,0.687251,80123
8,5571507,2024-09-01,365.0,283.5,0.287478,81.5,0.357738,80022
9,5432208,2024-09-01,283.0,215.5,0.313225,67.5,0.523482,80104


### Price reduction activity

In [28]:
# 
price_reductions_next = (
    list_events_with_data.copy(deep=True)
    .loc[:, ['parcl_property_id', 'event_date', 'event_name', 'price']]
    # discuss if we need to filter out this event
    .query('event_name != "PENDING_SALE"')
    # sort by parcl property id and date
    .sort_values(['parcl_property_id', 'event_date'], ascending=[False, True])
    .rename(columns={'event_date':'event_date_next', 'event_name':'event_name_next', 'price':'price_next'})
)
print(len(price_reductions_next), len(list_events_with_data))

173658 180165


In [29]:
price_reductions = (
    list_events_with_data.copy(deep=True)
    # Sort the DataFrame by 'parcl_property_id' (descending) and 'event_date' (ascending).
    .sort_values(['parcl_property_id', 'event_date'], ascending=[False, True])
    # Select relevant columns for further processing.
    .loc[:, ['parcl_property_id', 'event_date', 'event_name', 'price', 'latitude', 'longitude', 'property_type']]
    # Merge the DataFrame with 'price_reductions_next' to add information about subsequent price changes.
    .merge(price_reductions_next, on='parcl_property_id', how='left')
    # Filter rows where 'event_date' is different from 'event_date_next' and price has changed.
    .query('event_date != "event_date_next" & price != price_next')
    # Calculate the difference in days between 'event_date_next' and 'event_date'.
    .assign(day_diff=lambda df: (df['event_date_next'] - df['event_date']).dt.days)
    # Sort by 'parcl_property_id' and 'day_diff' to identify the largest time gap between events for each property.
    .sort_values(['parcl_property_id', 'day_diff'], ascending=[False, False])
    # Group by 'parcl_property_id' to keep only the first observation (with the largest day difference) for each property.
    .groupby('parcl_property_id').first()
    # Reset the index to make 'parcl_property_id' a regular
    .reset_index()  # Reset the index to make 'parcl_property_id' a regular column again.
    .assign(price_diff = lambda df: df['price_next'] - df['price'],
            price_diff_pct=lambda df: (df['price_next'] - df['price']) / df['price'])
    
)



In [30]:
# get the price differences
print(price_reductions['price_diff_pct'].median())
price_reductions = (
    price_reductions
    .query('price_diff_pct <-0.0')
    .query('price_diff_pct >= -0.8')
    .query('event_date >= "2024-08-01"& event_date <= "2024-09-01"')
)

-0.037267080745341616


In [31]:
# Load the data (replace this with your actual data loading method)
mbox_token = 'pk.eyJ1IjoiZGF0YXdyZXN0bGVyLXBhcmNsbGFicyIsImEiOiJjbHZ2bTRidGUxdndtMndvNnI5eGY5dDVoIn0.wXcsWmRjcDAlutloLezm5Q'

# Calculate min and max values for color scale
min_val = price_reductions['price_diff_pct'].min()
max_val = price_reductions['price_diff_pct'].max()

# Ensure that most negative values are mapped to bright red
heatmap = go.Densitymapbox(
    lat=price_reductions['latitude'],
    lon=price_reductions['longitude'],
    z=price_reductions['price_diff_pct'],
    radius=7,
    # Adjusted color scale to correctly map values (deep red for most negative)
     colorscale=[
        [0, '#000000'],          # Black
        [0.05, '#003380'],       # Black
        [0.25, '#ff0000'],       # Bright red for the most negative
        [0.35, '#f46d43'],       # Orange-red
        [0.45, '#e67300'],       # Orange-yellow
        [0.55, '#ff944d'],       # Lighter orange
        [0.65, '#ffe6cc'],       # Light orange-yellow
        [0.95, '#ffffff'],       # Very light yellow
        [1, '#ccffcc']           # White for the least negative
    ],
    zmin=min_val,
    zmax=max_val,
    hovertext=price_reductions['price_diff_pct'].apply(lambda x: f'{x:.2f}%'),
    hoverinfo='text',
    colorbar=dict(
        title=dict(
            text="Price Change (%)",
            font=dict(size=14)
        ),
        tickformat='.2%',  # Format to percentage
        thickness=10,      # Reduce thickness for a narrower colorbar
        len=0.3,           # Shorten the length of the colorbar
        x=0.82,            # Move colorbar closer to the center of the map
        y=0.5,             # Center colorbar vertically
        yanchor="middle",
        bgcolor='rgba(255,255,255,0.8)',  # Slightly transparent for better visibility
        bordercolor='rgba(0,0,0,0.5)',
        borderwidth=1
    )
)

# Create figure
fig = go.Figure(heatmap)

# Update layout
fig.update_layout(
    mapbox_accesstoken=mbox_token,
    mapbox=dict(
        center=dict(lat=price_reductions['latitude'].median(), lon=price_reductions['longitude'].median()),
        style="dark",
        zoom=9
    ),
    height=CHART_HEIGHT,
    width=CHART_WIDTH,  # Adjust width to make space for the colorbar
    title=dict(
        text=f"Listings in {market_name} with Price Reductions (%)",
        font=dict(size=24, family="Arial", weight='bold', color='white'),
        y=0.98,
        x=0.5,
        xanchor='center'
    ),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    font=dict(family="Arial", size=14, color="#ffffff"),  # White font for contrast
    margin=dict(l=0, r=0, t=40, b=0)  # Remove margin on right side
)

fig.add_layout_image(
    create_labs_logo_dict(color='white', x=0.2)
)

# Show the figure
fig.show()

# Save the figure
save_figure(fig, save_path=f'{ROOT_DIR}/map_active_listings_{market_name}_with_price_reductions.png', 
            width=CHART_WIDTH, height=CHART_HEIGHT)


### Share of Total Inventory by Property Type

In [32]:
# Define all types of property types to analyze supply data for each type.
property_types = ['SINGLE_FAMILY', 'CONDO', 'TOWNHOUSE']
df_supply = []

# Loop through each property type to retrieve supply data.
for property_type in property_types:
    # Retrieve the supply data (for-sale inventory) for the market, filtered by property type.
    # The 'start_date' sets the beginning of the data period, and 'auto_paginate=True' ensures that all pages of data are retrieved.
    supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
        parcl_ids=[market_for_analysis_id],  # The ID of the market to analyze.
        auto_paginate=True,  # Automatically paginate to retrieve all available data.
        start_date=start_date,  # Start date for the inventory data.
        property_type=property_type  # Filter data by the current property type in the loop.
    )
    # Append the retrieved supply data to the 'df_supply' list for later concatenation or analysis.
    df_supply.append(supply)


data: {'parcl_id': ['2899750'], 'start_date': '2022-09-01', 'property_type': 'SINGLE_FAMILY'}, params: {}
data: {'parcl_id': ['2899750'], 'start_date': '2022-09-01', 'property_type': 'CONDO'}, params: {}
data: {'parcl_id': ['2899750'], 'start_date': '2022-09-01', 'property_type': 'TOWNHOUSE'}, params: {}


In [33]:
# Concatenate all supply DataFrames for different property types into a single DataFrame.
df_supply_by_property = pd.concat(df_supply)

# Merge the combined supply DataFrame with 'supply_df' to bring in overall inventory data for each market and date.
df_supply_by_property = (
    df_supply_by_property
    .merge(
        supply_df[['parcl_id', 'date', 'for_sale_inventory']],  # Select relevant columns from 'supply_df'.
        on=['parcl_id', 'date'],  # Merge on market ID ('parcl_id') and 'date'.
        suffixes=('', '_all_properties')  # Add suffix to avoid column name conflicts after merging.
    )
    # Calculate the share of inventory for each property type relative to the total inventory of all property types.
    .assign(share_of_inventory=lambda df: df['for_sale_inventory'] / df['for_sale_inventory_all_properties'])
)

# Display the first few rows of the resulting DataFrame to verify the data.
df_supply_by_property.head()


,parcl_id,date,for_sale_inventory,property_type,for_sale_inventory_all_properties,share_of_inventory
0,2899750,2024-09-30,6965,SINGLE_FAMILY,13321,0.522859
1,2899750,2024-09-23,7109,SINGLE_FAMILY,13594,0.522951
2,2899750,2024-09-16,7106,SINGLE_FAMILY,13609,0.522154
3,2899750,2024-09-09,7158,SINGLE_FAMILY,13644,0.524626
4,2899750,2024-09-02,7166,SINGLE_FAMILY,13606,0.526679


In [34]:
# create chart
chart_max_date = df_supply_by_property['date'].max()
chart_max_date = chart_max_date.strftime('%B %d, %Y')

fig = px.line(
    df_supply_by_property,
    x='date',
    y='share_of_inventory',
    color='property_type',
    line_group='property_type',
    labels={'share_of_inventory': '% of Inventory for Sale'},
    title=f'Share of inventory by property type for {market_name} ({chart_max_date})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(df_supply_by_property['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = df_supply_by_property[
        (df_supply_by_property['property_type'] == trace.name) &
        (df_supply_by_property['date'] == latest_date)
    ]['share_of_inventory'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='Share of Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)
save_figure(fig, save_path=f'{ROOT_DIR}/share_intentory_property_type_{market_name}.png',
            width=CHART_WIDTH, height=CHART_WIDTH)
fig.show()

### Share of inventory that is coming from new construction

In [35]:
total_listings = []  # List to store DataFrames of new listings for each property type.
total_listings_new = []  # List to store DataFrames of new construction listings for each property type.
processed = []  # List to store processed DataFrames that merge new listings and new construction data.

# Loop through each specified property type to retrieve and process data.
for property_type in ['SINGLE_FAMILY', 'CONDO', 'TOWNHOUSE']:

    # Retrieve data on new listings for sale from the market metrics client.
    new_listings = client.market_metrics.housing_event_counts.retrieve(
        parcl_ids=market_for_analysis_id,  # Market ID for analysis.
        property_type=property_type  # Filter data by the current property type in the loop.
    )

    # Retrieve data on new construction listings from the new construction metrics client.
    new_construction = client.new_construction_metrics.housing_event_counts.retrieve(
        parcl_ids=market_for_analysis_id,  # Market ID for analysis.
        property_type=property_type  # Filter data by the current property type in the loop.
    )

    # Rename the column in the new construction DataFrame for clarity.
    new_construction = new_construction.rename(
        columns={'new_listings_for_sale': 'new_construction_new_listings_for_sale'}
    )

    # Merge 'new_listings' and 'new_construction' DataFrames on 'parcl_id' and 'date'.
    df = (
        new_listings
        .merge(
            new_construction[['parcl_id', 'date', 'new_construction_new_listings_for_sale']],
            on=['parcl_id', 'date']
        )
        # Calculate the percentage of new construction listings out of total new listings.
        .assign(pct_new_construction=lambda df: df['new_construction_new_listings_for_sale'] / df['new_listings_for_sale'])
    )

    # Append the retrieved new listings and new construction data to their respective lists.
    total_listings.append(new_listings)
    total_listings_new.append(new_construction)
    # Append the processed DataFrame (merged and with calculated percentage) to the 'processed' list.
    processed.append(df)


data: {'parcl_id': ['2899750'], 'property_type': 'SINGLE_FAMILY'}, params: {}
data: {'parcl_id': ['2899750'], 'property_type': 'SINGLE_FAMILY'}, params: {}
data: {'parcl_id': ['2899750'], 'property_type': 'CONDO'}, params: {}
data: {'parcl_id': ['2899750'], 'property_type': 'CONDO'}, params: {}
data: {'parcl_id': ['2899750'], 'property_type': 'TOWNHOUSE'}, params: {}
data: {'parcl_id': ['2899750'], 'property_type': 'TOWNHOUSE'}, params: {}


In [36]:
# Concatenate all processed DataFrames into a single DataFrame and perform further processing.
processed_df = (
    pd.concat(processed)  # Combine all DataFrames in the 'processed' list into one DataFrame.
    .sort_values(['property_type', 'date'], ascending=[False, True])  # Sort by 'property_type' and 'date'.
    .query('date >= "2022-09-01"')  # Filter rows to include only data from September 2022 onwards.
)

In [37]:
# create three different dataframes for each property type
processed_df_sf = processed_df.query('property_type == "SINGLE_FAMILY"')
processed_df_condo = processed_df.query('property_type == "CONDO"')
processed_df_townhouse = processed_df.query('property_type == "TOWNHOUSE"')

In [38]:
# create the chaart for single family
chart_max_date = processed_df_sf['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

# Create the stacked bar chart
fig = px.bar(processed_df_sf, 
             x='date', 
             y='pct_new_construction', 
             color='property_type', 
             barmode='group', 
             title=f'Percent of Supply Coming New Construction {market_name}: Single Family ({chart_max_date})',
             labels={'pct_new_construction': 'Percentage of inventory from new construction', 
                     'clean_name': 'Market'},
             color_discrete_sequence=['#ebba34'])

# Update the legend names
for trace in fig.data:
    if trace.name == 'New Construction':
        trace.name = 'New Construction'
    elif trace.name == 'Investors':
        trace.name = 'Investors'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='% of New Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())
save_figure(fig, save_path=f'{ROOT_DIR}/share_intentory_new_construction_{market_name}_sfh.png',
            width=CHART_WIDTH, height=CHART_WIDTH)

fig.show()

In [39]:
# create the chart for condos
chart_max_date = processed_df_condo['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

# Create the stacked bar chart
fig = px.bar(processed_df_condo, 
             x='date', 
             y='pct_new_construction', 
             color='property_type', 
             barmode='group', 
             title=f'Percent of Supply Coming New Construction {market_name}: Condos ({chart_max_date})',
             labels={'pct_new_construction': 'Percentage of inventory from new construction', 
                     'clean_name': 'Market'},
             color_discrete_sequence=['#ebba34'])

# Update the legend names
for trace in fig.data:
    if trace.name == 'New Construction':
        trace.name = 'New Construction'
    elif trace.name == 'Investors':
        trace.name = 'Investors'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='% of New Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())
save_figure(fig, save_path=f'{ROOT_DIR}/share_intentory_new_construction_{market_name}_condos.png',
            width=CHART_WIDTH, height=CHART_WIDTH)

fig.show()

In [40]:
# now lets do the townhouses
chart_max_date = processed_df_townhouse['date'].max()
chart_max_date = chart_max_date.strftime('%B, %Y')

# Create the stacked bar chart
fig = px.bar(processed_df_townhouse, 
             x='date', 
             y='pct_new_construction', 
             color='property_type', 
             barmode='group', 
             title=f'Percent of Supply Coming New Construction {market_name}: Townhouses({chart_max_date})',
             labels={'pct_new_construction': 'Percentage of inventory from new construction', 
                     'clean_name': 'Market'},
             color_discrete_sequence=['#ebba34'])

# Update the legend names
for trace in fig.data:
    if trace.name == 'New Construction':
        trace.name = 'New Construction'
    elif trace.name == 'Investors':
        trace.name = 'Investors'

fig.update_layout(
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': style_config['title_font']
    },
    xaxis=dict(
        title_text='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis'],
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color']),
        # showticklabels=False
    ),
    yaxis=dict(
        title_text='% of New Inventory',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    legend_title_text='',
    autosize=False,
    width=1600,
    height=800,
    title_font=dict(size=24),
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18),
    legend_title_font=dict(size=14),
    legend_font=dict(size=12),
    legend=dict(
        x=style_config['legend_x'],
        y=style_config['legend_y'],
        xanchor=style_config['legend_xanchor'],
        yanchor=style_config['legend_yanchor'],
        font=style_config['legend_font'],
        bgcolor='rgba(0, 0, 0, 0)'
    ),
)

fig.add_layout_image(create_labs_logo_dict())
save_figure(fig, save_path=f'{ROOT_DIR}/share_intentory_new_construction_{market_name}_townhouses.png',
            width=CHART_WIDTH, height=CHART_WIDTH)

fig.show()

### Price Cuts by Property Type

In [41]:
price_cuts_by_property_type_next = (
    # Create a deep copy of 'list_events_with_data' to prevent changes to the original DataFrame.
    list_events_with_data.copy(deep=True)
    # Sort the DataFrame by 'parcl_property_id' and 'event_date' in ascending order.
    .sort_values(['parcl_property_id', 'event_date'], ascending=[True, True])
    # Assign new columns to track the sequence of events for each property:
    # - 'index_of_dates' is the cumulative count of events for each 'parcl_property_id'.
    # - 'index_of_dates_next_r' is the cumulative count offset by one to track the next event.
    .assign(
        index_of_dates=lambda df: df.groupby('parcl_property_id').cumcount() + 1,
        index_of_dates_next_r=lambda df: df.groupby('parcl_property_id').cumcount() + 2
    )
    # Rename columns to reflect that they correspond to the next event in the sequence.
    .rename(columns={
        'event_date': 'event_date_next',
        'event_name': 'event_name_next',
        'price': 'price_next',
        'index_of_dates': 'index_of_dates_'
    })
    # Select a subset of columns relevant for tracking the next event details and index.
    .loc[:, ['parcl_property_id', 'event_date_next', 'event_name_next', 'price_next', 'index_of_dates_', 'index_of_dates_next_r']]
)


In [42]:
# Print the number of rows in the original 'list_events_with_data' DataFrame.
print(f"Number of events before processing: {len(list_events_with_data)}")

# Create a DataFrame that analyzes price cuts by property type by merging the data with itself.
price_cuts_by_property_type = (
    # Create a deep copy of 'list_events_with_data' to avoid modifying the original data.
    list_events_with_data.copy(deep=True)
    # Sort by 'parcl_property_id' and 'event_date' to facilitate tracking event sequences.
    .sort_values(['parcl_property_id', 'event_date'], ascending=[True, True])
    # Assign new columns to track event sequences:
    # 'index_of_dates' is a sequential index of events for each 'parcl_property_id'.
    # 'index_of_dates_next' is offset by one to track the subsequent event.
    .assign(
        index_of_dates=lambda df: df.groupby('parcl_property_id').cumcount() + 1,
        index_of_dates_next=lambda df: df.groupby('parcl_property_id').cumcount() + 2
    )
    # Merge with 'price_cuts_by_property_type_next' to align each event with its "next" event.
    .merge(
        price_cuts_by_property_type_next, 
        left_on=['parcl_property_id', 'index_of_dates_next'],  # Merge on 'parcl_property_id' and the "next" index.
        right_on=['parcl_property_id', 'index_of_dates_'],  # Match with the index in the subsequent event DataFrame.
        how='left'
    )
    # Filter rows where both 'event_date' and 'price' differ from the subsequent event's values.
    .query('event_date != event_date_next & price != price_next')
    # Calculate the difference in days between the event and its subsequent event.
    .assign(day_diff=lambda df: (df['event_date_next'] - df['event_date']).dt.days)
    # Sort by 'parcl_property_id' and 'event_date' to maintain event order.
    .sort_values(['parcl_property_id', 'event_date'])
    # Drop rows where 'event_date_next' is NaN, as they represent incomplete event sequences.
    .dropna(subset=['event_date_next'])
)

# Print the number of rows in 'price_cuts_by_property_type' to show the count after processing.
print(f"Number of price cut events after processing: {len(price_cuts_by_property_type)}")




Number of events before processing: 180165
Number of price cut events after processing: 81802


In [43]:
# Calculate and create variables of interest related to price cuts.
price_cuts_by_property_type_final = (
    price_cuts_by_property_type.copy(deep=True)  # Create a deep copy to prevent changes to the original DataFrame.
    .assign(
        # Calculate the number of days between the current event and the subsequent price cut event.
        days_between_price_cuts=lambda df: (df['event_date_next'] - df['event_date']).dt.days,
        # Calculate the absolute difference in price between the two events.
        price_diff=lambda df: df['price_next'] - df['price'],
        # Calculate the percentage difference in price between the two events.
        pct_price_diff=lambda df: (df['price_next'] - df['price']) / df['price'],
        # Convert the event date to a biweekly timestamp for potential grouping/analysis on a biweekly basis.
        event_date_biweekly=lambda df: df['event_date_next'].dt.to_period('2W').dt.to_timestamp(),
        # Convert the event date to a monthly timestamp for potential grouping/analysis on a monthly basis.
        event_date_monthly=lambda df: df['event_date_next'].dt.to_period('M').dt.to_timestamp()
    )
    # Filter the DataFrame to keep only rows where the percentage price difference is negative (i.e., price cuts).
    .query('pct_price_diff < 0')
)

In [53]:
# Group the data by 'property_type' and monthly event date to calculate median statistics for each month.
monthly_price_cuts_property = (
    price_cuts_by_property_type_final.groupby(['property_type', 'event_date_monthly'])
    .agg({
        'price_diff': 'median',  # Calculate the median price difference for each property type and month.
        'pct_price_diff': 'median',  # Calculate the median percentage price difference for each property type and month.
        'days_between_price_cuts': 'median'  # Calculate the median days between price cuts for each property type and month.
    })
    .reset_index()  # Reset index to return 'property_type' and 'event_date_monthly' as columns.
)

# Group the data by 'property_type' and biweekly event date to calculate median statistics for each biweekly period.
biweekly_price_cuts_property = (
    price_cuts_by_property_type_final.groupby(['property_type', 'event_date_biweekly'])
    .agg({
        'price_diff': 'median',  # Calculate the median price difference for each property type and biweekly period.
        'pct_price_diff': 'median',  # Calculate the median percentage price difference for each property type and biweekly period.
        'days_between_price_cuts': 'median'  # Calculate the median days between price cuts for each property type and biweekly period.
    })
    .reset_index()  # Reset index to return 'property_type' and 'event_date_biweekly' as columns.
    .sort_values(['property_type', 'event_date_biweekly'])  # Sort by 'property_type' and biweekly date for ordered time series analysis.
    # Calculate a rolling average of the percentage price difference over 3 biweekly periods.
    .assign(
        ma_pct_price_diff=lambda df: df.groupby('property_type')['pct_price_diff']
                                      .transform(lambda x: x.rolling(window=8).mean())
    )
    # drop nans for ma_pct_price_diff
    .dropna(subset=['ma_pct_price_diff'])
)


In [45]:
# create chart for monthly price moves
chart_max_date = monthly_price_cuts_property['event_date_monthly'].max()
chart_max_date = chart_max_date.strftime('%B %d, %Y')

fig = px.line(
    monthly_price_cuts_property,
    x='event_date_monthly',
    y='pct_price_diff',
    color='property_type',
    line_group='property_type',
    labels={'pct_price_diff': 'Median Inventory Price Reduction as Percent of Last Price'},
    title=f'Price Reduction by Property Type for {market_name} ({chart_max_date})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(monthly_price_cuts_property['event_date_monthly'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each property_type
    last_y_value = monthly_price_cuts_property[
        (monthly_price_cuts_property['property_type'] == trace.name) &
        (monthly_price_cuts_property['event_date_monthly'] == latest_date)
    ]['pct_price_diff'].values[0]
    
    # Add the annotation for each trace without checking for overlap
    annotations.append(dict(
        x=latest_date,
        y=last_y_value,
        xref='x',
        yref='y',
        text=trace.name,
        showarrow=False,
        xanchor='left',
        font=dict(size=12)  # Adjust the font size if needed
    ))
    y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='Price cuts by property type',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.2%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

save_figure(fig, save_path=f'{ROOT_DIR}/price_cuts_monthly_intentory_property_type_{market_name}.png',
            width=CHART_WIDTH, height=CHART_HEIGHT)
fig.show()
    

In [54]:
# create chart for biweekly price moves
chart_max_date = biweekly_price_cuts_property['event_date_biweekly'].max()
chart_max_date = chart_max_date.strftime('%B %d, %Y')

fig = px.line(
    biweekly_price_cuts_property,
    x='event_date_biweekly',
    y='ma_pct_price_diff',
    color='property_type',
    line_group='property_type',
    labels={'ma_pct_price_diff': 'Median Inventory Price Reduction as Percent of Last Price'},
    title=f'Price Reduction by Property Type for {market_name} ({chart_max_date})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(biweekly_price_cuts_property['event_date_biweekly'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each property_type
    last_y_value = biweekly_price_cuts_property[
        (biweekly_price_cuts_property['property_type'] == trace.name) &
        (biweekly_price_cuts_property['event_date_biweekly'] == latest_date)
    ]['ma_pct_price_diff'].values[0]
    
    # Add the annotation for each trace without checking for overlap
    annotations.append(dict(
        x=latest_date,
        y=last_y_value,
        xref='x',
        yref='y',
        text=trace.name,
        showarrow=False,
        xanchor='left',
        font=dict(size=12)  # Adjust the font size if needed
    ))
    y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='Price cuts by property type',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.2%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)

save_figure(fig, save_path=f'{ROOT_DIR}/price_cuts_biweekly_intentory_property_type_{market_name}.png',
            width=CHART_WIDTH, height=CHART_HEIGHT)
fig.show()


In [47]:
# inspect the data
#price_cuts_by_property_type.sort_values('event_date', ascending=True)
price_cuts_by_property_type.query('parcl_property_id == 213109813')

,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,...,zip_parcl_id,property_type,index_of_dates,index_of_dates_next,event_date_next,event_name_next,price_next,index_of_dates_,index_of_dates_next_r,day_diff
159064,213109813,2023-07-06,LISTING,PENDING_SALE,689400.0,0.0,1,NaN,None,0,...,5596238,SINGLE_FAMILY,2,3,2023-08-15,PRICE_CHANGE,670000.0,3.0,4.0,40.0
159066,213109813,2023-08-18,LISTING,RELISTED,670000.0,0.0,1,NaN,None,0,...,5596238,SINGLE_FAMILY,4,5,2023-08-19,PRICE_CHANGE,692400.0,5.0,6.0,1.0
159067,213109813,2023-08-19,LISTING,PRICE_CHANGE,692400.0,0.0,1,NaN,None,0,...,5596238,SINGLE_FAMILY,5,6,2023-08-23,PRICE_CHANGE,660000.0,6.0,7.0,4.0


### Real time price check

In [48]:
START_DATE = '2020-03-01'

# Retrieve price feed data for the market and the USA to analyze price trends.
pf_ids = [market_for_analysis_id, us_parcl_id]
sales_price_feeds = client.price_feed.price_feed.retrieve(
     parcl_ids=pf_ids,
     start_date=START_DATE,
     limit=1000,  # expand the limit to 1000, these are daily series
     auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)


data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2020-03-01', 'limit': 1000}, params: {'limit': 1000}


In [49]:
# Show percent change for sales price feeds relative to the first value after 2020-03-01

chart_pf = (
    sales_price_feeds.copy(deep=True)
    # Sort the data by date
    .sort_values('date')
    
    # Select relevant columns for further processing
    .loc[:, ['date', 'parcl_id', 'price_feed']]
    
    # Merge the current data with the first value for each 'parcl_id' on '3/1/2020'
    .merge(
        sales_price_feeds
        .loc[lambda df: df['date'] == '2020-03-01', ['parcl_id', 'price_feed']]
        .rename(columns={'price_feed': 'start'}),
        on='parcl_id'
    )
    
    # Calculate the percentage change relative to the start value
    .assign(
        pct_change=lambda df: (df['price_feed'] - df['start']) / df['start']
    )
    
    # Merge the data with the markets DataFrame to add clean market names
    .merge(markets[['parcl_id', 'name']], on='parcl_id')
    # Create Clean Name column for better visualization
    .assign(state = lambda df: df['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0]),
            clean_name =lambda df: df.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1))
    # replace name for USA
    .assign(clean_name=lambda df: np.where(df['parcl_id'] == us_parcl_id, 'USA', df['clean_name']))   
)


In [50]:
# create chart
chart_max_date = chart_pf['date'].max()
chart_max_date = chart_max_date.strftime('%B %d, %Y')
print(chart_max_date)

fig = px.line(
    chart_pf,
    x='date',
    y='pct_change',
    color='clean_name',
    line_group='clean_name',
    labels={'pct_change': '% Change'},
    title=f'% Change in Pricefeed since the Start of the Pandemic ({chart_max_date})'
)

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name == 'USA':
        trace.update(
            line=dict(color='red', width=4),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(chart_pf['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = chart_pf[
        (chart_pf['clean_name'] == trace.name) &
        (chart_pf['date'] == latest_date)
    ]['pct_change'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.02 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
        create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% Change',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=80, b=40),
    title={
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    },
    annotations=annotations  # Add annotations
)
save_figure(fig, save_path=f'{ROOT_DIR}/realtime_pct_change_home_values_since_covid_line_chart_{market_name}.png',
            width=CHART_WIDTH, height=CHART_HEIGHT)
fig.show()

October 06, 2024
